In [1]:
from IPython import parallel as p

In [2]:
pc = p.Client()

In [3]:
pc.ids

[0, 1, 2, 3, 4, 5]

In [4]:
view = pc.load_balanced_view()

In [7]:
%%px
import numpy as np
import pandas as pd
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
wyears = np.load('/Volumes/data/Theo/projects/Budyko_vic/wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [8]:
import pandas as pd
import numpy as np
import glob

dates = pd.read_pickle('./timecode.pcl')
wyears = np.load('./wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [9]:
def smbf(fl):
    lat = float(fl.split('_')[-2]) # extract latitude from file name
    lon = float(fl.split('_')[-1]) # extract longitude from file name

    # load the fluxes data
    flux = pd.read_table(fl, sep='\t', names=fluxes_columns)
    flux.index = pd.DatetimeIndex(dates)
    
    sm1 = flux.sm1.mean()
    sm2 = flux.sm2.mean()
    sm3 = flux.sm3.mean()
    bf = flux.BF.mean()
    
    return lat,lon,sm1,sm2,sm3,bf

In [10]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df')

In [11]:
files.head()

,exists,flux,forcing,lat,lon,indexer
0,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.03125,1
1,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.09375,2
2,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.15625,3
3,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.21875,4
4,True,/Users/barnhatb/research/vic/fluxes/ascii/lati...,/Users/barnhatb/research/vic/forcing/ascii/lat...,25.03125,-100.28125,5


In [15]:
smbf(files.flux[270900])

(50.28125,
 -117.28125,
 18.874508889459275,
 71.0174373374401,
 121.56040100530458,
 1.7804348448614817)

In [17]:
res = view.map(smbf,files.flux)

In [26]:
n = len(files)
m = res.progress
t = res.elapsed/60./60.

texp = (n/m)*t
trem = texp-t


print round(texp,2),'Total hours'
print round(trem,2), 'Hours remaining'

5.1 Total hours
0.0 Hours remaining


In [21]:
len(files)

309673

In [20]:
res.elapsed/60.

4.798091383333333

In [27]:
res.ready()

True

In [28]:
lat,lon,sm1,sm2,sm3,bf = zip(*res.result)

In [29]:
np.savez_compressed('./smbf.npz',lat=lat,lon=lon,sm1=sm1,sm2=sm2,sm3=sm3,bf=bf)

In [ ]:
(res.elapsed/60./60.)